In [39]:
import numpy as np
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.exceptions import SpotifyException
import time
from tqdm import tqdm

In [3]:
cli_cred_manager = SpotifyClientCredentials(client_id="3be9acd2dbcb4943b4573e5659d7f6ba",client_secret="53e274002b594989858aa743b4e81fc3")
sp = spotipy.Spotify(client_credentials_manager=cli_cred_manager)

---
# Crawl songs with genre

In [ ]:
genre_list = ["pop", "rock", "jazz", "classical", "blues", "hip-hop", "reggae", "electronic", "acoustic"]

In [ ]:
track_name = []
artist_name = []
album_name = []
genre = []
duration_ms = []
popularity = []
explicit = []
track_id = []
artist_id = []
track_popularity = []
artist_popularity = []
artist_genre = []

for g in genre_list:
    # Max 50 ID / API call
    for i in range(0,1000,50):
        q = 'genre:'+str(g)
        
        genre_results = sp.search(q=q, type='track', limit=50,offset=i)
        time.sleep(0.5) # sleep for weaks

        for i, t in enumerate(genre_results['tracks']['items']):
            track_name.append(t['name'])
            artist_name.append(t['artists'][0]['name'])
            album_name.append(t['album']['name'])
            genre.append(g)
            duration_ms.append(t['duration_ms'])
            popularity.append(t['popularity'])
            explicit.append(t['explicit'])
            track_id.append(t['id'])
            artist_id.append(t['artists'][0]['id'])
            track_popularity.append(t["popularity"])


In [26]:
df = pd.DataFrame({'track_id' : track_id,'track_name':track_name,'artist_id':artist_id, 'artist_name':artist_name, 'popularity' : track_popularity,
                   'album_name':album_name,'genre':genre,'duration_ms':duration_ms,
                   'popularity':popularity,'explicit':explicit
                   })

In [54]:
non_dupp_df = df.drop_duplicates(subset=['track_name','artist_name'])


In [63]:
dupes = non_dupp_df.groupby(['artist_name','track_name'], as_index=True).size()
len(dupes[dupes > 1])

0

In [62]:
non_dupp_df.groupby("genre").count()

,track_id,track_name,artist_id,artist_name,popularity,album_name,duration_ms,explicit
genre,,,,,,,,
acoustic,952,952,952,952,952,952,952,952
blues,840,840,840,840,840,840,840,840
classical,952,952,952,952,952,952,952,952
electronic,949,949,949,949,949,949,949,949
hip-hop,839,839,839,839,839,839,839,839
jazz,959,959,959,959,959,959,959,959
pop,968,968,968,968,968,968,968,968
reggae,879,879,879,879,879,879,879,879
rock,849,849,849,849,849,849,849,849


# Crawl Audio Features

In [ ]:
def audio_feature_collector(track_id_lst):
    audio_features = []
    batchsize = 100

    # tqdm for progress bar
    for i in tqdm(range(0,len(track_id_lst),batchsize)):
        batch = track_id_lst[i:i+batchsize]

        feature_results = sp.audio_features(batch)
        time.sleep(0.5)
        for track in feature_results:
            if track is not None:
                audio_features.append(track)

    df = pd.DataFrame.from_dict(data=audio_features,orient='columns')
    
    return df.rename(columns={'id':'track_id'})


audio_feats = audio_feature_collector(non_dupp_df["track_id"])

100%|██████████| 82/82 [01:01<00:00,  1.33it/s]


In [57]:
audio_feats


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,track_id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.521,0.592,6,-7.777,0,0.0304,0.3080,0.000000,0.1220,0.535,157.969,audio_features,2plbrEY59IikOBgBGLjaoe,spotify:track:2plbrEY59IikOBgBGLjaoe,https://api.spotify.com/v1/tracks/2plbrEY59Iik...,https://api.spotify.com/v1/audio-analysis/2plb...,251668,3
1,0.747,0.507,2,-10.171,1,0.0358,0.2000,0.060800,0.1170,0.438,104.978,audio_features,6dOtVTDdiauQNBQEDOtlAB,spotify:track:6dOtVTDdiauQNBQEDOtlAB,https://api.spotify.com/v1/tracks/6dOtVTDdiauQ...,https://api.spotify.com/v1/audio-analysis/6dOt...,210373,4
2,0.700,0.582,11,-5.960,0,0.0356,0.0502,0.000000,0.0881,0.785,116.712,audio_features,0WbMK4wrZ1wFSty9F7FCgu,spotify:track:0WbMK4wrZ1wFSty9F7FCgu,https://api.spotify.com/v1/tracks/0WbMK4wrZ1wF...,https://api.spotify.com/v1/audio-analysis/0WbM...,218424,4
3,0.674,0.907,3,-4.086,1,0.0640,0.1010,0.000000,0.2970,0.721,112.964,audio_features,5G2f63n7IPVPPjfNIGih7Q,spotify:track:5G2f63n7IPVPPjfNIGih7Q,https://api.spotify.com/v1/tracks/5G2f63n7IPVP...,https://api.spotify.com/v1/audio-analysis/5G2f...,157280,4
4,0.769,0.722,11,-5.485,0,0.0507,0.0584,0.000003,0.1110,0.570,119.973,audio_features,1Es7AUAhQvapIcoh3qMKDL,spotify:track:1Es7AUAhQvapIcoh3qMKDL,https://api.spotify.com/v1/tracks/1Es7AUAhQvap...,https://api.spotify.com/v1/audio-analysis/1Es7...,256000,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8182,0.658,0.273,10,-10.020,0,0.0309,0.7290,0.000727,0.1020,0.420,115.498,audio_features,1jO5kYMz6I9skWKu8Q9iS7,spotify:track:1jO5kYMz6I9skWKu8Q9iS7,https://api.spotify.com/v1/tracks/1jO5kYMz6I9s...,https://api.spotify.com/v1/audio-analysis/1jO5...,209600,4
8183,0.486,0.710,4,-5.501,1,0.0429,0.0425,0.000008,0.1080,0.438,145.821,audio_features,46QrTPAuu0iJHnECJKIWWH,spotify:track:46QrTPAuu0iJHnECJKIWWH,https://api.spotify.com/v1/tracks/46QrTPAuu0iJ...,https://api.spotify.com/v1/audio-analysis/46Qr...,272509,4
8184,0.451,0.512,9,-10.616,0,0.0563,0.9030,0.824000,0.1410,0.584,97.145,audio_features,6y9VXK7MuqlCzu7OvVGHTs,spotify:track:6y9VXK7MuqlCzu7OvVGHTs,https://api.spotify.com/v1/tracks/6y9VXK7MuqlC...,https://api.spotify.com/v1/audio-analysis/6y9V...,284200,4
8185,0.467,0.855,4,-5.375,1,0.0515,0.1740,0.000009,0.1100,0.565,135.813,audio_features,7AZAGSn2B1APKe2XBbzB2G,spotify:track:7AZAGSn2B1APKe2XBbzB2G,https://api.spotify.com/v1/tracks/7AZAGSn2B1AP...,https://api.spotify.com/v1/audio-analysis/7AZA...,135320,4


In [58]:
audio_feats_final = audio_feats.drop(columns=["type", "uri", "track_href", "duration_ms"])

In [59]:
merged_df = pd.merge(non_dupp_df, audio_feats_final, on="track_id")

In [60]:
merged_df

,track_id,track_name,artist_id,artist_name,popularity,album_name,genre,duration_ms,explicit,danceability,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,analysis_url,time_signature
0,2plbrEY59IikOBgBGLjaoe,Die With A Smile,1HY2Jd0NmPuamShAr6KMms,Lady Gaga,100,Die With A Smile,pop,251667,False,0.521,...,-7.777,0,0.0304,0.3080,0.000000,0.1220,0.535,157.969,https://api.spotify.com/v1/audio-analysis/2plb...,3
1,6dOtVTDdiauQNBQEDOtlAB,BIRDS OF A FEATHER,6qqNVTkY8uBg9cP3Jd7DAH,Billie Eilish,97,HIT ME HARD AND SOFT,pop,210373,False,0.747,...,-10.171,1,0.0358,0.2000,0.060800,0.1170,0.438,104.978,https://api.spotify.com/v1/audio-analysis/6dOt...,4
2,0WbMK4wrZ1wFSty9F7FCgu,"Good Luck, Babe!",7GlBOeep6PqTfFi59PTUUN,Chappell Roan,94,"Good Luck, Babe!",pop,218423,False,0.700,...,-5.960,0,0.0356,0.0502,0.000000,0.0881,0.785,116.712,https://api.spotify.com/v1/audio-analysis/0WbM...,4
3,5G2f63n7IPVPPjfNIGih7Q,Taste,74KM79TiuVKeVCqs8QtB0B,Sabrina Carpenter,94,Short n' Sweet,pop,157279,False,0.674,...,-4.086,1,0.0640,0.1010,0.000000,0.2970,0.721,112.964,https://api.spotify.com/v1/audio-analysis/5G2f...,4
4,1Es7AUAhQvapIcoh3qMKDL,Timeless (with Playboi Carti),1Xyo4u8uXC1ZmMpatF05PJ,The Weeknd,92,Timeless,pop,256000,True,0.769,...,-5.485,0,0.0507,0.0584,0.000003,0.1110,0.570,119.973,https://api.spotify.com/v1/audio-analysis/1Es7...,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8182,1jO5kYMz6I9skWKu8Q9iS7,This Love Is Over (with The Pariah Dogs),6DoH7ywD5BcQvjloe9OcIj,Ray LaMontagne,39,God Willin' & The Creek Don't Rise (with The P...,acoustic,209600,False,0.658,...,-10.020,0,0.0309,0.7290,0.000727,0.1020,0.420,115.498,https://api.spotify.com/v1/audio-analysis/1jO5...,4
8183,46QrTPAuu0iJHnECJKIWWH,Hold My Heart,2Sqr0DXoaYABbjBo9HaMkM,Sara Bareilles,36,Kaleidoscope Heart,acoustic,272613,False,0.486,...,-5.501,1,0.0429,0.0425,0.000008,0.1080,0.438,145.821,https://api.spotify.com/v1/audio-analysis/46Qr...,4
8184,6y9VXK7MuqlCzu7OvVGHTs,Stairway to Heaven,7vX3cMVyW8gtDA4y855ynF,Rodrigo y Gabriela,32,Rodrigo y Gabriela,acoustic,284200,False,0.451,...,-10.616,0,0.0563,0.9030,0.824000,0.1410,0.584,97.145,https://api.spotify.com/v1/audio-analysis/6y9V...,4
8185,7AZAGSn2B1APKe2XBbzB2G,Far Behind,0mXTJETA4XUa12MmmXxZJh,Eddie Vedder,46,Into The Wild (Music For The Motion Picture),acoustic,135320,False,0.467,...,-5.375,1,0.0515,0.1740,0.000009,0.1100,0.565,135.813,https://api.spotify.com/v1/audio-analysis/7AZA...,4


In [66]:
merged_df.to_csv("../data/train_data_with_genre.csv", index=False)

# Crawl Artist genre

In [69]:
unique_artist_ids = np.unique(merged_df["artist_id"])

In [78]:
def get_artist_genres(unique_artist_ids):
    end = unique_artist_ids.shape[0]
    step = 50
    artist_genres = {}

    for i in tqdm(range(0, end, step)):
        end_ = min(i + step, end)
        # print(min(i + 50, 125))
        artists_idx = unique_artist_ids[i:end_]
        res_artists = sp.artists(artists_idx)

        for artist in res_artists["artists"]:
            name = artist["name"]
            id = artist["id"]
            genres = artist["genres"]
            artist_genres[id] = ",".join(genres)

    return pd.DataFrame.from_dict(data=artist_genres, orient="index")

artist_genre_df = get_artist_genres(unique_artist_ids)

100%|██████████| 44/44 [00:38<00:00,  1.16it/s]


In [ ]:
artist_genre_df = artist_genre_df.reset_index()

In [88]:
artist_genre_df.columns = ["artist_id", "artist_genres"]
artist_genre_df

,artist_id,artist_genres
0,00FQb4jTyendYWaN8pK0wa,"art pop,pop"
1,00Nnsh2ykMBYnCDsSFGXSY,"jazz guitar,smooth jazz"
2,00OLN5o0Ba1W6KkjRi5jWd,"lithuanian edm,lithuanian electronic"
3,00XhexlJEXQstHimpZN910,"reggaeton,trap latino,urbano latino"
4,00me4Ke1LsvMxt5kydlMyU,"latin hip hop,reggaeton,trap latino,urbano latino"
...,...,...
2148,7yk35uHNQclPXFGFoTU44w,"alternative hip hop,hardcore hip hop,hip hop,j..."
2149,7ymrIsmp4vE7vAtcS12J4D,"modern alternative rock,rebel blues"
2150,7zk65sKwJHv61CsjCr8PVw,background jazz
2151,7zrkALJ9ayRjzysp4QYoEg,"electronica,future garage,indie soul"


In [89]:
merged_df_final = pd.merge(merged_df, artist_genre_df, on="artist_id")

In [90]:
merged_df_final

,track_id,track_name,artist_id,artist_name,popularity,album_name,genre,duration_ms,explicit,danceability,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,analysis_url,time_signature,artist_genres
0,2plbrEY59IikOBgBGLjaoe,Die With A Smile,1HY2Jd0NmPuamShAr6KMms,Lady Gaga,100,Die With A Smile,pop,251667,False,0.521,...,0,0.0304,0.30800,0.000000,0.1220,0.535,157.969,https://api.spotify.com/v1/audio-analysis/2plb...,3,"art pop,dance pop,pop"
1,5R8dQOPq8haW94K7mgERlO,Poker Face,1HY2Jd0NmPuamShAr6KMms,Lady Gaga,76,The Fame,pop,237200,False,0.851,...,1,0.0787,0.11900,0.000002,0.1210,0.774,119.001,https://api.spotify.com/v1/audio-analysis/5R8d...,4,"art pop,dance pop,pop"
2,0SiywuOBRcynK0uKGWdCnn,Bad Romance,1HY2Jd0NmPuamShAr6KMms,Lady Gaga,82,The Fame Monster (Deluxe Edition),pop,294573,True,0.695,...,1,0.0363,0.00314,0.000053,0.0842,0.714,119.007,https://api.spotify.com/v1/audio-analysis/0Siy...,4,"art pop,dance pop,pop"
3,2x7MyWybabEz6Y6wvHuwGE,Just Dance,1HY2Jd0NmPuamShAr6KMms,Lady Gaga,74,The Fame,pop,241933,False,0.822,...,0,0.0311,0.02640,0.000043,0.1810,0.745,118.990,https://api.spotify.com/v1/audio-analysis/2x7M...,4,"art pop,dance pop,pop"
4,11BKm0j4eYoCPPpCONAVwA,Bloody Mary,1HY2Jd0NmPuamShAr6KMms,Lady Gaga,73,Born This Way (Special Edition),pop,244760,False,0.591,...,0,0.0291,0.01070,0.000002,0.1130,0.440,99.998,https://api.spotify.com/v1/audio-analysis/11BK...,4,"art pop,dance pop,pop"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8182,0bxev1OTsBrnuQYFRLYZft,Careless Whisper,5ZtuBRCHqXY3bSD2BRSYNC,Alexandr Misko,41,Beyond the Box,acoustic,249865,False,0.605,...,0,0.0411,0.56500,0.899000,0.0939,0.673,77.148,https://api.spotify.com/v1/audio-analysis/0bxe...,4,acoustic guitar cover
8183,4IBlIZtU6v7SnCNf7kdCgx,Three Little Birds - Acoustic,3cGb8b9Zwvka0MWD5RyMDI,Richard Watts,52,Three Little Birds (Acoustic),acoustic,165405,False,0.551,...,1,0.0418,0.51600,0.000000,0.1210,0.800,73.388,https://api.spotify.com/v1/audio-analysis/4IBl...,4,acoustic cover
8184,5JPS38wdSCxAW8AYfGe6Ei,Good Fucking Bye,7qVz2CTVWQynqvLaaxZXw6,Matt Skiba,35,Matt Skiba & Kevin Seconds Split,acoustic,152066,False,0.629,...,1,0.0263,0.19800,0.000006,0.0948,0.798,131.988,https://api.spotify.com/v1/audio-analysis/5JPS...,4,"acoustic punk,chicago punk"
8185,0EzykG4pYlmmGTihakWGRL,It's Nobody's Fault but Mine,5kO4xdEKLuHHHPreu3UmkZ,Blind Willie Johnson,37,Dark Was The Night (Mojo Workin'- Blues For Th...,acoustic,188533,False,0.676,...,1,0.0390,0.98000,0.000074,0.1100,0.483,83.901,https://api.spotify.com/v1/audio-analysis/0Ezy...,4,"acoustic blues,blues,country blues,gospel blue..."


In [93]:
merged_df_final = merged_df_final[['track_id', 'track_name', 'artist_id', 'artist_name', 'popularity',
       'album_name', 'genre', 'artist_genres', 'duration_ms', 'explicit', 'danceability',
       'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'analysis_url',
       'time_signature']]

